In [2]:
# Preamble 
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

import numpy as np
from astropy.table import Table, join, hstack, vstack

In [50]:
irafdr60 = Table.read('../../auxiliary_information/dr6.0.fits')
# We need the stars to have Gaia source_id
irafdr60 = irafdr60[(np.where(irafdr60['source_id'] > 0)[0])]

In [98]:
galahdr3 = Table.read('../../../GALAH_DR3/catalogs/GALAH_DR3_main_allspec_v2.fits')
galahdr3['source_id'] = galahdr3['dr3_source_id']
# We need the stars to have Gaia source_id
galahdr3 = galahdr3[(np.where(galahdr3['source_id'] > 0)[0])]

# Accuracy estimators:
## VESTA, Gaia FGK Benchmark Stars, Bensby+2014 Stars, Nissen+Schuster2010

In [113]:
gbs = np.array([# VESTA

# From Jofre et al. (2018) RNAAS, 2, 152
[131216002601003,'mu Leo'],
[131216003201003,'HD 84937'],
[131220004401099,'ksi Hya'],
[140209004701151,'eps Vir'],
[140209005201151,'HD 122563'],
[140308003401188,'18 Sco'],
[140708003401078,'mu Ara'],
[140708005801203,'bet Hyi'],
[140708006401203,'tau Cet'],
[140709001901194,'eta Boo'],
[140709004401117,'gam Sge'],
[140710000801284,'HD 107328'],
[140710001701284,'ksi Hya'],
[140710002501284,'18 Sco'],
[140710003901284,'18 Sco'],
[140710006601104,'alf Cet'],
[140710007601032,'eps Eri'],
[140710008301032,'del Eri'],
[140710009001032,'HD 22879'],
[140806005301134,'eps For'],
[140808000901102,'HD 140283'],
[141101003801339,'Procyon'],
[141102003801353,'alf Tau'],
[141102004001353,'HD 49933'],
[141231005201176,'HD 84937'],
[150204002101256,'mu Leo'],
[150210005801171,'Arcturus'],
[150210006401171,'eta Boo'],
[150405000901378,'Sun'],
# From Nissen & Schuster (2010) A&A, 511, 10
[140111004101214, 'HD76932'],
[170205005901355, 'HD121004'],
[171102005001013, 'G112-43'],
# From Bensby et al. (2014) A&A, 562, 71
[140712005201149, 'HIP 117880'],
[140114005001165, 'HIP 23555'],
[140114004701165, 'HIP 23941'],
[140114005301165, 'HIP 24829'],
[131119001701221, 'HIP 30503'],
[140712005601149, 'HIP 3142'],
[140111004101214, 'HIP 44075'],
[140114005801165, 'HIP 72673'],
[140114006101165, 'HIP 75181'],
[140807001701174, 'HIP 86796'],
[140807002101174, 'HIP 87523'],
[140807002501174, 'HIP 96124'],
[140807003101174, 'HIP 96536'],
[140713003901146, 'HIP 99240']
])

sobject_ids = [['#sobject_id','GBS/Bensby/Nissen']]
available = 0
not_available = 0

for sobject_id,comment in gbs:
    if int(sobject_id) in irafdr60['sobject_id']:
        sobject_ids.append([sobject_id,comment])
        available += 1
    else:
        sobject_ids.append(['#'+sobject_id,comment])
        not_available += 1
        
print('available: ',available,' not available: ',not_available)
np.savetxt('gaia_fgk_benchmark_stars',np.array(sobject_ids),fmt='%s')
dates = np.unique([gbs[x,0][:6] for x in range(len(gbs))])
print('Unique dates:')
dates

available:  2  not available:  44
Unique dates:


array(['131119', '131216', '131220', '140111', '140114', '140209',
       '140308', '140708', '140709', '140710', '140712', '140713',
       '140806', '140807', '140808', '141101', '141102', '141231',
       '150204', '150210', '150405', '170205', '171102'], dtype='<U6')

# Open Clusters

In [69]:
cantat_gaudin = Table.read('../../auxiliary_information/CantatGaudin_2020_AandA_640_1.fits')
cantat_gaudin['source_id'] = cantat_gaudin['GaiaDR2']

In [119]:
oc_dr3 = join(galahdr3,cantat_gaudin,keys='source_id',join_type='inner')

In [120]:
oc_dr60 = join(irafdr60,cantat_gaudin,keys='source_id',join_type='inner')

In [140]:
all_oc_sobject_ids = np.concatenate(([np.array(oc_dr3['sobject_id']),np.array(oc_dr60['sobject_id'])]))
all_oc_cluster_names = np.concatenate(([np.array(oc_dr3['Cluster']),np.array(oc_dr60['Cluster'])]))
unique_oc_sobject_ids, unique_indices = np.unique(all_oc_sobject_ids,return_index=True)
clusters_unique = all_oc_cluster_names[unique_indices]

sobject_ids = [['#sobject_id','cluster']]
available = 0
not_available = 0

for (sobject_id,comment) in zip(unique_oc_sobject_ids,clusters_unique):
    if int(sobject_id) in irafdr60['sobject_id']:
        sobject_ids.append(str(sobject_id))
        available += 1
    else:
        sobject_ids.append('#'+str(sobject_id))
        not_available += 1
        
print('available: ',available,' not available: ',not_available)
np.savetxt('open_cluster_stars',sobject_ids,fmt='%s')
dates = np.unique([gbs[x,0][:6] for x in range(len(gbs))])
print('Unique dates:')
dates

available:  4146  not available:  618
Unique dates:


array(['131119', '131216', '131220', '140111', '140114', '140209',
       '140308', '140708', '140709', '140710', '140712', '140713',
       '140806', '140807', '140808', '141101', '141102', '141231',
       '150204', '150210', '150405', '170205', '171102'], dtype='<U6')

# Globular Clusters

In [149]:
vasiliev_baumgardt = Table.read('../../auxiliary_information/VasilievBaumgardt_2021_MNRAS_505_5978_8GCs_70percent.fits')

In [150]:
gc_dr3 = join(galahdr3,vasiliev_baumgardt,keys='source_id',join_type='inner')

In [151]:
gc_dr60 = join(irafdr60,vasiliev_baumgardt,keys='source_id',join_type='inner')

In [153]:
all_gc_sobject_ids = np.concatenate(([np.array(gc_dr3['sobject_id']),np.array(gc_dr60['sobject_id'])]))
all_gc_cluster_names = np.concatenate(([np.array(gc_dr3['cluster']),np.array(gc_dr60['cluster'])]))
unique_gc_sobject_ids, unique_indices = np.unique(all_gc_sobject_ids,return_index=True)
clusters_unique = all_gc_cluster_names[unique_indices]

sobject_ids = [['#sobject_id','cluster']]
available = 0
not_available = 0

for (sobject_id,comment) in zip(unique_gc_sobject_ids,clusters_unique):
    if int(sobject_id) in irafdr60['sobject_id']:
        sobject_ids.append(str(sobject_id))
        available += 1
    else:
        sobject_ids.append('#'+str(sobject_id))
        not_available += 1
        
print('available: ',available,' not available: ',not_available)
np.savetxt('globular_cluster_stars',sobject_ids,fmt='%s')
dates = np.unique([gbs[x,0][:6] for x in range(len(gbs))])
print('Unique dates:')
dates

available:  2626  not available:  1591
Unique dates:


array(['131119', '131216', '131220', '140111', '140114', '140209',
       '140308', '140708', '140709', '140710', '140712', '140713',
       '140806', '140807', '140808', '141101', '141102', '141231',
       '150204', '150210', '150405', '170205', '171102'], dtype='<U6')

# Asteroseismic Targets

In [154]:
zinn = Table.read('../../auxiliary_information/Zinn_Table2_eDR3_xmatch.fits')

In [155]:
zinn_dr3 = join(galahdr3,zinn,keys='source_id',join_type='inner')

In [156]:
zinn_dr60 = join(irafdr60,zinn,keys='source_id',join_type='inner')

In [157]:
all_sobject_ids = np.concatenate(([np.array(zinn_dr3['sobject_id']),np.array(zinn_dr60['sobject_id'])]))
all_numax = np.concatenate(([np.array(zinn_dr3['numax_mean']),np.array(zinn_dr60['numax_mean'])]))
unique_sobject_ids, unique_indices = np.unique(all_sobject_ids,return_index=True)
numax_unique = all_numax[unique_indices]

sobject_ids = [['#sobject_id','numax']]
available = 0
not_available = 0

for (sobject_id,comment) in zip(unique_sobject_ids,numax_unique):
    if int(sobject_id) in irafdr60['sobject_id']:
        sobject_ids.append(str(sobject_id))
        available += 1
    else:
        sobject_ids.append('#'+str(sobject_id))
        not_available += 1
        
print('available: ',available,' not available: ',not_available)
np.savetxt('asteroseismic_stars',sobject_ids,fmt='%s')
dates = np.unique([gbs[x,0][:6] for x in range(len(gbs))])
print('Unique dates:')
dates

available:  10464  not available:  951
Unique dates:


array(['131119', '131216', '131220', '140111', '140114', '140209',
       '140308', '140708', '140709', '140710', '140712', '140713',
       '140806', '140807', '140808', '141101', '141102', '141231',
       '150204', '150210', '150405', '170205', '171102'], dtype='<U6')